# LangChain

LangChain是构建大模型驱动的代理和应用程序的最简单的方式。

## Agent

Agent将LLM和Tool结合，能够对任务进行推理、决定哪些工具迭代的完成任务。

### 模型

Agent推理的核心是模型，LangChain支持静态模型和动态模型。静态模型指的是运行Agent时制定某个LLM；而动态模型则会根据运行时的状态和上下文环境，动态的选择合适的模型。

#### 静态模型

In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI

load_dotenv()

# 工具函数
@tool
def multiply(a: str, b: str) -> str:
    """
    将两个字符串转换为浮点数并计算它们的乘积。
    
    Args:
        a (str): 第一个数字的字符串表示
        b (str): 第二个数字的字符串表示
        
    Returns:
        str: 格式化的字符串，显示乘法表达式和结果
        
    Example:
        >>> multiply("2.5", "3.2")
        "2.5 * 3.2 = 8.0"
        
    Note:
        如果输入字符串无法转换为浮点数，将抛出ValueError异常
    """
    return f"{float(a)} * {float(b)} = {float(a) * float(b)}"

llm = ChatOpenAI(
    model="deepseek-chat", 
    api_key=os.getenv("OPENAI_API_KEY"), 
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0.1,
    max_tokens=1000,
)

agent = create_agent(
    model=llm,
    tools=[multiply]
)

print("开始流式输出：\n" + "="*50)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "计算3.5乘以2.8"}]}
):
    # 提取并格式化输出
    for key, value in chunk.items():
        if key == 'model':
            # 提取AI消息内容
            messages = value.get('messages', [])
            for msg in messages:
                if hasattr(msg, 'content'):
                    print(f"AI: {msg.content}")
        elif key == 'tools':
            # 提取工具消息内容
            messages = value.get('messages', [])
            for msg in messages:
                if hasattr(msg, 'content'):
                    print(f"工具调用: {msg.content}")
print("="*50 + "\n流式输出完成！")

#### 动态模型

动态模型在运行时根据当前状态和上下文选择。这支持复杂的路由逻辑和成本优化。

In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.tools import tool
from langchain_openai import ChatOpenAI

load_dotenv()

basic_model = ChatOpenAI(
    model="deepseek-chat", 
    api_key=os.getenv("OPENAI_API_KEY"), 
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0.1,
    max_tokens=1000,
)

advanced_model = ChatOpenAI(
    model="deepseek-reasoner", 
    api_key=os.getenv("OPENAI_API_KEY"), 
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0.1,
    max_tokens=1000,
)

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)


# 工具函数
@tool
def multiply(a: str, b: str) -> str:
    """
    将两个字符串转换为浮点数并计算它们的乘积。
    
    Args:
        a (str): 第一个数字的字符串表示
        b (str): 第二个数字的字符串表示
        
    Returns:
        str: 格式化的字符串，显示乘法表达式和结果
        
    Example:
        >>> multiply("2.5", "3.2")
        "2.5 * 3.2 = 8.0"
        
    Note:
        如果输入字符串无法转换为浮点数，将抛出ValueError异常
    """
    return f"{float(a)} * {float(b)} = {float(a) * float(b)}"

llm = ChatOpenAI(
    model="deepseek-chat", 
    api_key=os.getenv("OPENAI_API_KEY"), 
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0.1,
    max_tokens=1000,
)

agent = create_agent(
    model=llm,
    tools=[multiply],
    middleware=[dynamic_model_selection]
)

print("开始流式输出：\n" + "="*50)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "计算3.5乘以2.8"}]}
):
    # 提取并格式化输出
    for key, value in chunk.items():
        if key == 'model':
            # 提取AI消息内容
            messages = value.get('messages', [])
            for msg in messages:
                if hasattr(msg, 'content'):
                    print(f"AI: {msg.content}")
        elif key == 'tools':
            # 提取工具消息内容
            messages = value.get('messages', [])
            for msg in messages:
                if hasattr(msg, 'content'):
                    print(f"工具调用: {msg.content}")
print("="*50 + "\n流式输出完成！")

## Tool

工具赋予智能体执行操作的能力

### 定义工具并传给Agent使用

```python
from langchain.tools import tool
from langchain.agents import create_agent


@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    return f"Weather in {location}: Sunny, 72°F"

agent = create_agent(model, tools=[search, get_weather])
```

### 工具的错误处理

要自定义工具错误的处理方式，请使用 @wrap_tool_call 装饰器创建中间件

```python
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage


@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

agent = create_agent(
    model="gpt-4o",
    tools=[search, get_weather],
    middleware=[handle_tool_errors]
)
```

当工具失败时，智能体将返回一个包含自定义错误消息的 ToolMessage
```
[
    ...
    ToolMessage(
        content="Tool error: Please check your input and try again. (division by zero)",
        tool_call_id="..."
    ),
    ...
]
```

## 结构化输出

在某些情况下，您可能希望智能体以特定格式返回输出。LangChain 通过 response_format 参数提供结构化输出策略。

### 工具策略

ToolStrategy 使用人工工具调用来生成结构化输出。这适用于任何支持工具调用的模型。

```python
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_tool],
    response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')
```

### 提供者策略

ProviderStrategy 使用模型提供者原生的结构化输出生成。这更可靠，但仅适用于支持原生结构化输出的提供者（例如 OpenAI）。

```python
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model="gpt-4o",
    response_format=ProviderStrategy(ContactInfo)
)
```

## 中间件

中间件为在执行的不同阶段自定义智能体行为提供了强大的可扩展性。您可以使用中间件来

1. 在模型调用前处理状态（例如，消息截断、上下文注入）
2. 修改或验证模型的响应（例如，防护措施、内容过滤）
3. 使用自定义逻辑处理工具执行错误
4. 根据状态或上下文实现动态模型选择
5. 添加自定义日志、监控或分析

中间件无缝集成到智能体的执行图中，允许您在关键点拦截和修改数据流，而无需更改核心智能体逻辑。